Source reconstruction problem with given boundary data ($ICP_V$ and $ICP_{SAS}$) and smooth pressure assumption. Minimization problem:

$$
\min_{g} J(g) = \frac{\gamma_1}{2} \int_0^T \int_{\Gamma_V}(p(x,t) - ICP_V(x,t))^2 ds \, dt \\
+ \frac{\gamma_2}{2} \int_0^T \int_{\Gamma_{SAS}}(p(x,t) - ICP_{SAS}(x,t))^2 ds \, dt \\
+ \frac{\gamma_3}{2} \int_0^T \int_{\Omega} (\Delta p(x,t))^2 ds \, dt
$$

subject to the Darcy equation with Neumann boundary conditions and inital data:


$$
\begin{aligned}
c \frac{\partial p}{\partial t} + K \Delta p &= g \quad \text{ in } \Omega \times (0,T) \\
K \nabla p \cdot n &= 0 \quad  \text{ on } (\Gamma_V \cup \Gamma_{SAS}) \times (0,T)  \\
p &= 0 \quad \text { in } \Omega \times \{0 \}
\end{aligned}
$$

Since we can not directly compute the laplacian of the first order Lagrange solution of the pressure, we replace $\Delta p$  it with:

$$
\sum_E \int_E [ \nabla p] ds
$$

where $[\cdot]$ denotes the jump term on internal facets $E$. Now, we can solve the optimization problem by dolfin-adjoint and some gradient based optimization algorithm (e.g. L-BFGS, Newton-CG, ..)


In [ ]:
%load_ext autoreload
%autoreload 2
from braininversion.meshes import generate_flower_mesh, generate_doughnut_mesh
from fenics import *
import matplotlib.pyplot as plt
import numpy as np
from fenics_adjoint import *
from braininversion.DarcySolver import solve_darcy
from braininversion.Optimization import optimize_darcy_source, compute_minimization_target, update_expression_time
from braininversion.PlottingHelper import (plot_pressures_and_forces_timeslice, 
                            plot_pressures_and_forces_cross_section,
                            extract_cross_section, style_dict)
from braininversion.reconstructByLaplaceSolve import(solve_laplace_for_all_timesteps,
                                     compute_sources_from_pressures,
                                     solve_laplace)

In [ ]:
# time stepping
T = 1.2           # final time
num_steps = 10    # number of time steps
dt = T/ num_steps
times = np.linspace(dt, T, num_steps)

# material parameter
kappa = 1e-17 # permeability 15*(1e-9)**2
visc = 0.8*1e-3     # viscocity 
K = kappa/visc      # hydraulic conductivity
c = 2*1e-4          # storage coefficient
mmHg2Pa = 132.32
material_parameter = {"K":K, "c":c}

brain_radius = 0.1
ventricle_radius = brain_radius/4
n_petals = 8
N = 10
mesh, boundary_marker = generate_flower_mesh(brain_radius, n_petals, N)

f = 1
A = 2*mmHg2Pa
p_skull = Expression("A*sin(2*pi*f*t)", A=A,f=f,t=0,degree=2)
#p_ventricle = Expression("A*sin(2*pi*f*t)", A=A*0.8,f=f,t=0,degree=2)
p_ventricle = Constant(0.0)

x_coords = np.linspace(0*brain_radius, 1.2*brain_radius, 100)
slice_points = [Point(x, 0.0) for x in x_coords]

boundary_conditions = {1:{"Neumann":Constant(0.0)}, 2:{"Neumann":Constant(0.0)}}

alpha = 1e-5
def laplace(p):
    mesh = p.function_space().mesh()
    n = FacetNormal(mesh)
    return alpha*jump(grad(p), n)**2

    
minimization_target = {"ds": { 1: lambda x: (x - p_ventricle)**2,
                              2: lambda x: (x - p_skull)**2},
                       "dS":{"everywhere":laplace}}

time_dep_expr = [p_skull, p_ventricle]

In [ ]:
plt.figure(figsize=(9,7))
plot(mesh)
for p in slice_points:
    plt.scatter(p.array()[0],p.array()[1], s=20, color="red")

In [ ]:
# optimize Flower Mesh

# compute constant controls
res = optimize_darcy_source(mesh, material_parameter, times, minimization_target,
                            boundary_marker, boundary_conditions,
                            time_dep_expr=time_dep_expr, opt_solver="scipy",
                            control_args="constant"
                            )

opt_ctrls, opt_solution, initial_solution = res

# compute controls
res = optimize_darcy_source(mesh, material_parameter, times, minimization_target,
                            boundary_marker, boundary_conditions,
                            time_dep_expr=time_dep_expr,
                            opt_solver="moola_NewtonCG",
                            optimization_parameters={"jtol":1e-2,
                                                     "maxiter":100,
                                                     #"mem_lim":10
                                                    },
                            #control_args=["DG", 0],
                            initial_guess=opt_ctrls)

opt_ctrls, opt_solution, initial_solution = res

***Problem: Very slow convergence of iterative gradient descent methods!***
(slow propagation of information in the domain, only very few points have a nonzero gradient...)

***Shortcut: Compute optimal pressure:***

Assume that there extists an optimal pressure $p_{opt}$, such that:

$$
\begin{aligned}
K \Delta p_{opt} &= 0 \quad \quad &&\text{ in } \Omega \times (0,T) \\
 p_{opt} &= ICP \quad &&\text{ on } (\Gamma_V \cup \Gamma_{SAS}) \times (0,T)
\end{aligned}
$$

Then the corresponding source $g_{opt}$ can easily be computed:

$$
g_{opt} = c \frac{\partial p_{opt}}{\partial t} + \underbrace{K \Delta p}_{= 0} =  c \frac{\partial p_{opt}}{\partial t}
$$

and $g_{opt}$ minimizes the functional. \
**Note:** The same simplification probably also delivers good results for different regularization terms, since $K\Delta p$ is usually very small!

In [ ]:
laplace_bc = {1:{"Dirichlet":p_ventricle}, 2:{"Dirichlet":p_skull}}
         
laplace_pressures = solve_laplace_for_all_timesteps(mesh, boundary_marker, laplace_bc, 
                                                   times, time_dep_expr)

laplace_sources = compute_sources_from_pressures(laplace_pressures, c, dt)

laplace_solution = solve_darcy(mesh, laplace_sources, T, num_steps, K,
                              boundary_marker, boundary_conditions,
                              c=c)
laplace_solution = [s.copy() for s in laplace_solution]

In [ ]:
pressures = {"p_opt_const" : extract_cross_section(initial_solution, slice_points)/mmHg2Pa,
             "p_opt" : extract_cross_section(opt_solution, slice_points)/mmHg2Pa,
             "p_laplace":extract_cross_section(laplace_pressures, slice_points)/mmHg2Pa,}

forces = {"g_opt": extract_cross_section(opt_ctrls, slice_points),
          "g_laplace": extract_cross_section(laplace_sources, slice_points),
         }

style_dict["p_laplace"] = {"ls":":", "lw":3, "color":"firebrick"}
style_dict["f_laplace"] = {"ls":":", "lw":3, "color":"firebrick"}
style_dict["g_opt"] = {"ls":"-.", "lw":3, "color":"green"}
style_dict["g_opt"] = {"ls":"-.", "lw":3, "color":"green"}

In [ ]:
for i in [2,4,6,8]: 
    plot_pressures_and_forces_cross_section(pressures, forces, i, x_coords)
    plt.suptitle(f"t = {times[i]:.3f} s")

In [ ]:
for i in [40, 60 ,80]:
    plot_pressures_and_forces_timeslice(pressures, forces, i, times)
    plt.suptitle(f"Point: ({slice_points[i].x():.3f}, {slice_points[i].y():.3f})")

***Open Questions:***
* Implications of laplacian regularization term?
* Improvement of convergenve of the optimization problem (Preconditioning)